### Práctica de Formatos de datos: CSV y JSON

#### Ejercicio 1[5 puntos]
Considerar el archivo __Contaminación.csv__. El archivo contiene la información recogida por las estaciones de control de calidad del aire durante el año 2019.

Una descripción detallada de la estructura de los datos se puede encontrar en el siguiente enlace: https://datos.madrid.es/FWProjects/egob/Catalogo/MedioAmbiente/Aire/Ficheros/Interprete_ficheros_%20calidad_%20del_%20aire_global.pdf

La información contenida por columnas es la siguiente:
* __PROVINCIA__: Código numérico que representa la provincia. 
* __MUNICIPIO__: Código numérico que representa el municipio. 
* __ESTACIÓN__: Código numérico de hasta dos dígitos que representa la estación. Consultar ANEXO I de la documentación
* __MAGNITUD__: Código numérico de hasta dos dígitos que representa la magnitud.Consultar ANEXO II de la documentación
* __PUNTO MUESTREO__: Código numérico formado por 3 subcódigos. El primer código representa la estación, el segundo código representa la magnitud, y el tercer código representa la técnica de medida.
* __MES__: Mes de la toma de datos. Valor entre 1 y 12.
* __D01-D31__: Medidas tomadas desde el día D01 hasta el día D31 del mes correspondiente.
* __V01-V31__: Validez de las medidas tomadas desde el día D01 hasta el día D31 del mes correspondiente. Puede valer V(Válido), N(No válido), no se conoce(vacío) o bien otra cadena(se considera incorrecto).


Se pide crear un programa que realice las siguientes operaciones:
* Leer los datos del archivo __Contaminación.csv__
* Transformar los datos leidos de manera que aquellos valores que no sean válidos ("N") su valor sea sustituido por 0.
* Obtener y  mostrar por pantalla para cada tipo de contaminante (columna Magnitud) y para cada estación (columna ESTACIÓN), la suma total de la contaminación que se ha producido durante el año 2019, es decir:

|Estación|Contaminante1| Contaminante2| Contaminante3| ....Contaminanten|
|----|-------------|-------------|-------------|-------------|
|Estación1|Subtotal11 |--- |--- |Subtotal1n|  
|Estación2|--- |--- |--- |--- | 
|   ...|--- |--- |--- |--- |     
|Estaciónn|Subtotal1n |--- |--- |Subtotalnn| 
     
* Generar un documento __Salida.csv__ donde cada línea del documento contiene la siguiente información:
    Estación1, Contaminante1,Subtotal1, Contaminante2,Subtotal2,......................,Contaminanten,Subtotaln
    
    Estaciónm, Contaminante1,Subtotal1, Contaminante2,Subtotal2,......................,Contaminanten,Subtotaln
    
Tanto en lo que mostréis por pantalla como lo que escribáis en el archivo usad los nombres de los contaminantes y de las estaciones que aparecen en los anexos de la documentación indicada más arriba.
    

Para leer los archivos usa un código como el siguiente:

In [103]:
import csv
import re

csvarchivo = open('Contaminación.csv',encoding="utf8",errors='ignore')
entrada = csv.reader(csvarchivo, delimiter=";")
contador1 = 0
verifica = 'V'
patron = re.compile(verifica)
indicesValorar = []
indicesDatos = []
magnitudes=[]
estaciones=[]
estacionActual = None
magnitudActual = None
contadorActual = 0
contadoresEstacionActual = []
tablaDatos = []
cogidos = False
for i in entrada:
    if cogidos == False:
        for j in i:
            if(patron.match(j)):
                indicesValorar.append(contador1)
                indicesDatos.append(contador1-1)
            contador1+=1
        cogidos=True
    else:
        for indice in indicesValorar:
            if i[indice]=="N":
                i[indice-1] = 0
        if estacionActual == None:
            estacionActual = i[2]
            magnitudActual = i[3]
            for dato in indicesDatos:
                contadorActual+=float(i[dato])
        elif estacionActual != i[2]:
            contadoresEstacionActual.append([magnitudActual, contadorActual])
            tablaDatos.append([estacionActual, contadoresEstacionActual])
            estacionActual = i[2]
            magnitudActual = i[3]
            contadorActual = 0
            for dato in indicesDatos:
                contadorActual+=float(i[dato])
            contadoresEstacionActual = []
        else:
            if magnitudActual == i[3]:
                for dato in indicesDatos:
                    contadorActual+=float(i[dato])
            else:
                contadoresEstacionActual.append([magnitudActual, contadorActual])
                magnitudActual = i[3]
                contadorActual = 0
                for dato in indicesDatos:
                    contadorActual+=float(i[dato])
tablaDatos.append([estacionActual, contadoresEstacionActual])

tablaFinal = ""
for i in tablaDatos:
    tablaFinal += "Estacion " + i[0]
    for j in i[1]:
        tablaFinal += " Contaminante " + j[0] + " Total acumulado " + str(j[1])
    tablaFinal += ' /n '
    
print(tablaFinal)
   


Estacion 4 Contaminante 1 Total acumulado 2016.0 Contaminante 6 Total acumulado 104.99999999999984 Contaminante 7 Total acumulado 6373.0 Contaminante 8 Total acumulado 10392.0 Contaminante 12 Total acumulado 20147.0 /n Estacion 8 Contaminante 1 Total acumulado 2098.0 Contaminante 6 Total acumulado 46.700000000000145 Contaminante 7 Total acumulado 4819.0 Contaminante 8 Total acumulado 12109.0 Contaminante 9 Total acumulado 2595.0 Contaminante 10 Total acumulado 5230.0 Contaminante 12 Total acumulado 19486.0 Contaminante 14 Total acumulado 12049.0 Contaminante 20 Total acumulado 632.9000000000001 Contaminante 30 Total acumulado 146.8000000000002 Contaminante 35 Total acumulado 113.50000000000009 Contaminante 42 Total acumulado 375.07000000000016 Contaminante 43 Total acumulado 352.2100000000001 Contaminante 44 Total acumulado 23.12 /n Estacion 11 Contaminante 7 Total acumulado 3417.0 Contaminante 8 Total acumulado 9288.0 Contaminante 12 Total acumulado 14549.0 Contaminante 20 Total acumu

#### Ejercicio 2[5 puntos]
Considerar los archivos  __Estaciones.csv__ y __Museos.json__ que incluye información sobre las estaciones de control de la calidad del aire y sobre los museos de Madrid respectivamente.

El objetivo de este ejercicio es utilizar la información obtenida en el ejercicio 1 de forma que para cada uno de los museos que aparece en el archivo dado realice las siguientes operaciones:
* Obtener las 3 estaciones más cercanas a cada museo.
* Obtener y mostrar por pantalla para cada museo la siguiente información donde ValorX es el subtotal del nivel de contaminación más alto de todos los contaminantes medidos en la estaciónX, y TipoContaminanteX es el tipo del contaminante cuyo valor es mostrado en la celda anterior.

|Museo|Estación1|TipoContaminante1| Estación2|TipoContaminante2| Estación3|TipoContaminante3|
|----|-------------|-------------|-------------|----|----|----|
|Museo1|Valor1 |Tipo1 |Valor2|Tipo2|Valor3|  Tipo3 |
|Museo2|--- |--- |--- |--- | --- | --- | 
|   ...|--- |--- |--- |--- | --- | --- |     
|Museon|Valor1 |Tipo1 |Valor2|Tipo2|Valor3|  Tipo3 |

* Generar un documento __Salida.json__ con la siguiente estructura:

Tanto en lo que mostréis por pantalla como lo que escribáis en el archivo usad los nombres de los contaminantes y de las estaciones que aparecen en los anexos de la documentación indicada más arriba.

Para leer los archivos usa un código como el siguiente:

In [ ]:
import json
leer = json.loads(open('Museos.json',encoding="utf8").read())
print(leer)

En los documentos __Estación.csv__ y __Museos.json__ aparece la información de geolocalización en forma de longitud y latitud tanto de las estaciones como de los museos. En este sentido, para calcular la distancia entre dos puntos dadas sus coordenadas se utilizará la siguiente función en Python:

#### Normas de entrega

* Fecha tope de entrega: 03/10/2019
* La entrega se realizará subiendo al campus virtual un notebook de Jupyter con la solución. El archivo tendrá como nombre FormatosI_GrupoX donde X será el número de grupo correspondiente.